In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
data = pd.read_csv('data/p4kreviews.csv', encoding='latin1')

In [11]:
data.head()
data.describe()

,Unnamed: 0,best,score
count,19555.000000,19555.000000,19555.000000
mean,9778.000000,0.053183,7.027446
std,5645.186593,0.224405,1.277544
min,1.000000,0.000000,0.000000
25%,4889.500000,0.000000,6.500000
50%,9778.000000,0.000000,7.300000
75%,14666.500000,0.000000,7.800000
max,19555.000000,1.000000,10.000000


# Data Prep

In [13]:
# -- might be interesting to do a separate analysis 

In [17]:
grouped_by_artist = data.groupby(['artist'])
grouped_by_artist

In [20]:
# Remove "Various Artists" since doesn't give us insight into a single artist.
reviews = data[data.artist != "Various Artists"]


# Exploration

# Visualization

# Analysis